# 0. Package Dependency

- [nltk](https://www.nltk.org)
- [sklearn](http://scikit-learn.org/stable/)

In [1]:
# Load packages
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import WordPunctTokenizer
import numpy as np
# Load data
trn_texts = open("trn-reviews.txt").read().strip().split("\n")
trn_labels = open("trn-labels.txt").read().strip().split("\n")
print("Training data ...")
print("%d, %d" % (len(trn_texts), len(trn_labels)))

dev_texts = open("dev-reviews.txt").read().strip().split("\n")
dev_labels = open("dev-labels.txt").read().strip().split("\n")
print("Development data ...")
print("%d, %d" % (len(dev_texts), len(dev_labels)))


Training data ...
40000, 40000
Development data ...
5000, 5000


In [3]:
glove = pd.read_csv("glove.6b.50d.txt", sep=' ', header=None, quotechar=None,  quoting=3)
dict = {}
for i, row in glove.iterrows():
    if len(dict)%50000 ==0:
        print(len(dict))
    dict[row[0]] = np.array(row.drop([0]))
    
    
wtrn_data = np.array([WordPunctTokenizer().tokenize(i.lower()) for i in trn_texts])
wdev_data = np.array([WordPunctTokenizer().tokenize(i.lower()) for i in dev_texts])

0
50000
100000
150000
200000
250000
300000
350000


In [4]:
def txt_rep(data):
    new_data = np.array([[0]*50])
    k = 0
    for i in data:
        addition = np.array([0]*50)
        count = 0
        for j in i:
            try:
                temp = dict[j]
                addition = np.sum([temp, addition], axis=0)
                count+=1
            except: pass
        count = max(1, count)
        try:
            new_data = np.append(new_data,[addition/count], axis=0)
        except:
            print(addition, count)
        if k%10000 == 0:
            print(k)
        k+=1
    return new_data

gtrn_data = txt_rep(wtrn_data)[1:]
gdev_data = txt_rep(wdev_data)[1:]
print('this block is done')

0
10000
20000
30000
0
this block is done


In [5]:
print(gtrn_data.shape)

(40000, 50)


In [6]:
from sklearn.linear_model import LogisticRegression


print(gtrn_data.shape)
# Define a LR classifier
classifier = LogisticRegression(solver="liblinear", multi_class="auto")
classifier.fit(gtrn_data, trn_labels)

# Measure the performance on training and dev data
print("Training accuracy = %f" % classifier.score(gtrn_data, trn_labels))
print("Dev accuracy = %f", classifier.score(gdev_data, dev_labels))

(40000, 50)
Training accuracy = 0.516325
Dev accuracy = %f 0.5464


In [7]:
choice = 1

if choice == 1:
    print("Preprocessing without any feature selection")
    vectorizer = CountVectorizer(lowercase=False)
    # vocab size 77166
elif choice == 2:
    print("Lowercasing all the tokens")
    vectorizer = CountVectorizer(lowercase=True)
    # vocab size 60610
else:
    raise ValueError("Unrecognized value: choice = %d" % choice)

trn_data = vectorizer.fit_transform(trn_texts)
print(trn_data.shape)
dev_data = vectorizer.transform(dev_texts)
print(dev_data.shape)

Preprocessing without any feature selection
(40000, 77166)
(5000, 77166)


In [8]:
from scipy.sparse import coo_matrix, hstack, csr_matrix

# dev_data = dev_data.toarray()
print(trn_data.shape, gtrn_data.shape)
# cdev_data = np.concatenate((dev_data.toarray(),gdev_data), axis=1)
# ctrn_data = np.concatenate((trn_data.toarray(),gtrn_data), axis=1)

A = csr_matrix(gtrn_data.astype(float))
ctrn_data = hstack([trn_data,A])
B = csr_matrix(gdev_data.astype(float))
cdev_data = hstack([dev_data,B])

# def comb(data, gdata):
#     new_data = np.array([[0]*77216])
#     k = 0
#     for i,j in zip(data,gdata):
#         if k%(n/10) ==0:
#             print (k)
#         if k == n:
#             print(k)
#             break
#         new_data = np.append(new_data, [np.append(i.toarray(), j )], axis=0)   
#         k+=1
#     return new_data
# ctrn_data = comb(trn_data, gtrn_data)[1:]    
# cdev_data = comb(dev_data, gdev_data)[1:]    

(40000, 77166) (40000, 50)


In [9]:
print(ctrn_data.shape)

(40000, 77216)


# 3. Logistic Regression

Please refer to the document of [_LogisticRegression_](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for the parameters of this function. 

In [10]:
from sklearn.linear_model import LogisticRegression


# Define a LR classifier
classifier = LogisticRegression(solver="liblinear", multi_class="auto")
classifier.fit(ctrn_data, trn_labels)

# Measure the performance on training and dev data
print("Training accuracy = %f" % classifier.score(ctrn_data, trn_labels))
print("Dev accuracy = %f", classifier.score(cdev_data, dev_labels))

Training accuracy = 0.941400
Dev accuracy = %f 0.6134


In [11]:
from sklearn.linear_model import LogisticRegression


# Define a LR classifier
classifier = LogisticRegression(solver="saga", multi_class="auto")
classifier.fit(ctrn_data, trn_labels)

# Measure the performance on training and dev data
print("Training accuracy = %f" % classifier.score(ctrn_data, trn_labels))
print("Dev accuracy = %f", classifier.score(cdev_data, dev_labels))

c:\users\ssjk\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy = 0.751075
Dev accuracy = %f 0.6394


In [12]:
from sklearn.linear_model import LogisticRegression


# Define a LR classifier
classifier = LogisticRegression(solver="sag", multi_class="auto")
classifier.fit(ctrn_data, trn_labels)

# Measure the performance on training and dev data
print("Training accuracy = %f" % classifier.score(ctrn_data, trn_labels))
print("Dev accuracy = %f", classifier.score(cdev_data, dev_labels))

Training accuracy = 0.792175
Dev accuracy = %f 0.6362


In [15]:
from sklearn.linear_model import LogisticRegression

# Define a LR classifier
classifier = LogisticRegression(solver="lbfgs", multi_class="auto")
classifier.fit(ctrn_data, trn_labels)

# Measure the performance on training and dev data
print("Training accuracy = %f" % classifier.score(ctrn_data, trn_labels))
print("Dev accuracy = %f", classifier.score(cdev_data, dev_labels))

c:\users\ssjk\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Training accuracy = 0.741375
Dev accuracy = %f 0.6272
